In [1]:
import string
import numpy as np
import re
import networkx
from tqdm import tqdm
import matplotlib.pyplot as plt
import networkx as nx
from collections import Counter
import itertools
from nltk.stem.porter import PorterStemmer

In [2]:
fhr = open('../datasets/amazon-meta.txt', 'r', encoding='utf-8', errors='ignore')


columns = ['Title', 'Categories', 'Group', 'SalesRank', 'TotalReviews', 'AvgRating']
MetaData = dict(zip(columns, [[] for i in columns]))


(Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff) = \
    ("", "", "", "", "", "", 0, 0, 0.0, 0, 0.0)
for line in tqdm(fhr):
    line = line.strip()
    # a product block started
    if(line.startswith("title")):
        Title = line[6:].strip()
        Title = ' '.join(Title.split())
    elif(line.startswith("group")):
        Group = line[6:].strip()
    elif(line.startswith("salesrank")):
        SalesRank = line[10:].strip()
    elif(line.startswith("categories")):
        ls = line.split()
        Categories = ' '.join((fhr.readline()).lower() for i in range(int(ls[1].strip())))
        Categories = re.compile('[%s]' % re.escape(string.digits+string.punctuation)).sub(' ', Categories)
    elif(line.startswith("reviews")):
        ls = line.split()
        TotalReviews = ls[2].strip()
        AvgRating = ls[7].strip()
    elif (line==""):
        try:
            MetaData['Title'].append(Title)
            MetaData['Categories'].append(' '.join(set(Categories.split())))
            MetaData['Group'].append(Group)
            MetaData['SalesRank'].append(int(SalesRank))
            MetaData['TotalReviews'].append(int(TotalReviews))
            MetaData['AvgRating'].append(float(AvgRating))
        except NameError as err:
            print(err)
            continue
        (Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, DegreeCentrality, ClusteringCoeff) = \
            ("", "", "", "", "", "", 0, 0, 0.0, 0, 0.0)
fhr.close()
titles = MetaData['Title']
del MetaData

12500875it [02:30, 82862.26it/s] 


In [3]:
import pandas as pd
from sklearn.decomposition import PCA
from pickle import load

def read_matrix_pickle():
    # 0:14 -- float64
    # 0:02 -- float32
    with open("../datasets/model.pickle", "rb") as f:
        mat = PCA(4).fit_transform(load(f))
        words = [i.split('_')[0] for i in load(f)]
        word_dict = dict(zip(words, range(len(words))))
        del words
    return mat, word_dict

w2v, word_dict = read_matrix_pickle()

In [4]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

# 3:28
matrix = np.zeros((len(titles), 4))
n_words_list = []
for i, title in tqdm(enumerate(titles), total=len(titles)):
    n_words = 0
    for word in title.split():
        word = porter_stemmer.stem(word)
        if word in word_dict:
            n_words += 1
            matrix[i] += w2v[word_dict[word]]
    if n_words > 0:
        matrix[i] = matrix[i] / n_words
    n_words_list.append(n_words)

100%|██████████| 548553/548553 [04:19<00:00, 2110.58it/s]


In [7]:
import numpy as np

zero_ind = np.array([i == 0 for i in n_words_list])
nonzero_sub = matrix[~zero_ind, :]
matrix[zero_ind, :] = nonzero_sub.mean(axis=0) +
                      np.random.randn(zero_ind.sum(), matrix.shape[1]) * nonzero_sub.std(axis=0)

In [10]:
from pickle import dump
dump(matrix, open("../tmp_files/title_submatrix", "wb"))

In [11]:
from collections import Counter

Counter(n_words_list)

Counter({0: 60622,
         1: 125751,
         2: 146761,
         3: 96854,
         4: 57037,
         5: 30909,
         6: 15767,
         7: 7806,
         8: 3652,
         9: 1781,
         10: 794,
         11: 419,
         12: 233,
         13: 79,
         14: 52,
         15: 17,
         16: 7,
         17: 8,
         20: 1,
         23: 1,
         26: 2})